In [32]:
import os
import numpy as np
import pandas as pd
import scipy.io.wavfile as wav
from sklearn.mixture import GaussianMixture
from python_speech_features import mfcc

def extract_features(audio_path, trim_length=None):
	sr, y = wav.read(audio_path)
	if trim_length:
		y = y[:trim_length * sr]  # multiplying by sr to convert seconds to samples
	mfccs = mfcc(y, sr, numcep=100)
	return mfccs

def train_gmm(features, n_components=3):
	gmm = GaussianMixture(n_components=n_components)
	gmm.fit(features)
	return gmm

def load_and_process_audio(file_path):
	features = extract_features(file_path)
	return features

def main():
	# Directory containing the training audio files
	directory = "../../real_audio/"
	# Load client profiles
	client_profiles = pd.read_csv("../../../client_profiles/real_match_name.csv")
	# Create a dictionary mapping rec_ids to file paths and client names
	audio_files = {row['rec_id']: (os.path.join(directory, row['rec_id'] + '.wav'), row['name'])
				for _, row in client_profiles.iterrows() if os.path.isfile(os.path.join(directory, row['rec_id'] + '.wav'))}

	gmms = {}
	for rec_id, (file_path, name) in audio_files.items():
		features = load_and_process_audio(file_path)
		gmm = train_gmm(features)
		gmms[rec_id] = gmm
	#save gmms in src/impersonator/gmms.npy
	# np.save('gmms.npy', gmms)

	# Load matched results to compare
	matched_results = pd.read_csv("filtered_matched_results.csv")

	# Compare results
	results = []
	directory = "../../../audio_clips"
	for _, row in matched_results.iterrows():
		rec_id = row['rec_id']
		actual_name = row['name']
		# Get the best match
		best_match = None
		features = load_and_process_audio(os.path.join(directory, rec_id + '.wav'))
		best_score = -np.inf
		for rec_id, gmm in gmms.items():
			score = gmm.score(features)
			if score > best_score:
				best_score = score
				best_match = rec_id
		rec_id = row['rec_id']
		print(best_score)
		best_match_name = audio_files[best_match][1]
		match_result = actual_name == best_match_name
		results.append((rec_id, actual_name, best_match_name, match_result))
		# print(f"Actual name: {actual_name}, Best match: {best_match_name}, Match result: {match_result}")


	df_results = pd.DataFrame(columns=['rec_id', 'is_impersonator'])
	for rec_id, actual_name, best_match_name, match_result in results:
		print(f"rec_id: {rec_id}, actual_name: {actual_name}, best_match_name: {best_match_name}, match_result: {match_result}")
		if match_result:
			df_results = df_results._append({'rec_id': rec_id, 'is_impersonator': 0}, ignore_index=True)
		else:
			df_results = df_results._append({'rec_id': rec_id, 'is_impersonator': 1}, ignore_index=True)
	#save results in src/impersonator/impersonator.csv
	df_results.to_csv('src/impersonator/impersonator.csv', index=False)

if __name__ == "__main__":
	main()



-76.60868006967293
-94.17680896268458
-92.11631950833596
-76.84557114970181


-92.37052712377714
-93.7435744485612
-74.5080388904029


-96.12396602916395
-97.58004989663898
-90.7260028118405


-97.47884025633356


-75.0940895797581
-88.68946330182278


...

-75.63585004817081
-80.25268766454971
-78.27733014396196


-76.69376454040096
-93.0437509873332
-98.65752911566221
-79.73324749320325


-76.92571746578162
-75.05500324002124


-74.17367312991138
-92.8324067193883
-94.15921523010363


-73.4867323294706
rec_id: 43QRG7SY14, actual_name: Elias Svensson, best_match_name: Fatima Zahra Mansouri, match_result: False
rec_id: YXAXYF3SXW, actual_name: Finn Murphy, best_match_name: Charlotte Leclerc, match_result: False
rec_id: ZVS4M3Y201, actual_name: Mia Anderson, best_match_name: Mia Anderson, match_result: True
rec_id: NZBALVR6CF, actual_name: Mia Anderson, best_match_name: Olivia Tremblay, match_result: False
rec_id: GHSWJ7IZ96, actual_name: Amina Kouri, best_match_name: Lara van den Berg, match_result: False
rec_id: N2VPD13IWV, actual_name: Olivia Tremblay, best_match_name: Charlotte Leclerc, match_result: False
rec_id: 656QZ2VWR4, actual_name: Jorge Castillo, best_match_name: Jorge Castillo, match_result: True
rec_id: UMM0YIUKNH, actual_name: Noah Zimmerman, best_match_name: Fatima Zahra Mansouri, match_result: False
rec_id: Z1BCC9U0E5, actual_name: Sofia Almeida, best_match_name: Lara van den Berg, match_result: False
rec_id: XDAOHC9OY5, actual_name: Mia Anderson, best